In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np
import gensim
import pickle
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!python --version


Python 3.9.16


In [ ]:
gensim.__version__

'4.3.1'

In [ ]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
# Load the word2vec model from disk
with open('/content/wv_model_25march.pickle', 'rb') as f:
    wv_model = pickle.load(f)


# Load the logistic regression model from disk
with open('/content/lr_model_25march.pickle', 'rb') as f:
    lr_model = pickle.load(f)

# # Load the word2vec model from disk
# with open('/content/wv_model_17_march.pickle', 'rb') as f:
#     wv_model17 = pickle.load(f)

# # Load the logistic regression model from disk
# with open('/content/lr_model_17march.pickle', 'rb') as f:
#     lr_model17 = pickle.load(f)

# with open('/content/rf_model.pickle', 'rb') as f:
#     rf_model = pickle.load(f)




In [ ]:
# Pre-process the text data
def preprocess_text(text):
    # Remove special characters and numbers
    text = gensim.utils.simple_preprocess(text)
    return text

In [ ]:
# Create the features
def get_vector(text):
    vector = np.zeros(100)
    count = 0
    for word in text:
        try:
            vector += wv_model.wv[word]
            count += 1
        except KeyError:
            pass
    if count != 0:
        vector /= count
    return vector

In [ ]:
# Get user input
text = input('Enter a text description: ')
text = preprocess_text(text)
vector = get_vector(text)
#predict_label(vector)

# Predict the label
prediction = lr_model.predict([vector])

# Output the label
print('The label is:', prediction[0])

Enter a text description: windsheild
The label is: All 4 wheel drive not working


In [ ]:
#import the test dataset to be tested
data = pd.read_excel('/content/test_reports.xlsx')
data

,Problem 1,Logic Tree Name
0,Not starting,No start
1,Not starting,No start
2,Not Crancking,No start
3,Engine Shaking,Misfire
4,Engine Stumbles,Misfire
...,...,...
137,Engine misfire or cylinder misfire,Misfire
138,No Heat from HVAC,No heat from hvac
139,AC Not Working,Ac not working
140,Engine is overheating,Engine overheating


In [ ]:
data['Problem 1'][2]

'Not Crancking'

In [ ]:
#bulk testing , appending to list then create a dataframe with the predictions
f_pre = []
for i in range(0,len(data['Logic Tree Name'])):
  text = data['Problem 1'][i]
  text = preprocess_text(text)
  vector = get_vector(text)
  prediction = rf_model.predict([vector])
  f_pre.append(prediction[0])
ser = pd.Series(f_pre)
data['rf_prediction_new'] = ser.values


In [ ]:
data

,Problem 1,Logic Tree Name,lr_prediction_17march,lr_prediction_new,rf_prediction_new
0,Not starting,No start,No start,No start,No start
1,Not starting,No start,No start,No start,No start
2,Not Crancking,No start,Windshield wipers inoperative,Windshield wipers inoperative,Windshield wipers inoperative
3,Engine Shaking,Misfire,Misfire,Misfire,Low power hard acceleration
4,Engine Stumbles,Misfire,Misfire,Misfire,Low power hard acceleration
...,...,...,...,...,...
137,Engine misfire or cylinder misfire,Misfire,Misfire,Misfire,Coolant leak
138,No Heat from HVAC,No heat from hvac,No heat from hvac,No heat from hvac,Ac not working
139,AC Not Working,Ac not working,Ac not working,Ac not working,Ac not working
140,Engine is overheating,Engine overheating,Engine overheating,Engine overheating,Abs light on


In [ ]:
#creating an excel file of the results
data.to_excel('test_compareing_models_25march.xlsx')